<a href="https://colab.research.google.com/github/jasmine-yolo/Python-practice/blob/main/P_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import tqdm
import matplotlib.pyplot as plt
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, mean_absolute_error

In [ ]:
from google.colab import drive

# mount the Google drive as a path that you can work on 
drive.mount('/drive')

df_0 = pd.read_csv('/drive/My Drive/Colab Notebooks/Data/smalldata.csv', parse_dates=['DATE'])

Mounted at /drive


In [ ]:
df_0.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77836 entries, 0 to 77835
Data columns (total 102 columns):
 #    Column           Non-Null Count  Dtype         
---   ------           --------------  -----         
 0    Unnamed: 0       77836 non-null  float64       
 1    DATE             77836 non-null  datetime64[ns]
 2    permno           77836 non-null  float64       
 3    RET              77836 non-null  float64       
 4    mvel1            77836 non-null  float64       
 5    prc              77836 non-null  float64       
 6    SHROUT           77836 non-null  float64       
 7    beta             77836 non-null  float64       
 8    betasq           77836 non-null  float64       
 9    chmom            77836 non-null  float64       
 10   dolvol           77836 non-null  float64       
 11   idiovol          77836 non-null  float64       
 12   indmom           77836 non-null  float64       
 13   mom1m            77836 non-null  float64       
 14   mom6m            778

In [ ]:
df=df_0.dropna(axis=0, how='any')

df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
#set DATE as index
df.set_index('DATE', inplace=True)
df.sort_values('DATE', inplace=True)

In [ ]:
df['lead(RET,1)'] = df['RET'].shift(-1)
df.drop(columns=['RET'],inplace=True)
df.dropna(axis=0, how='any')

,permno,mvel1,prc,SHROUT,beta,betasq,chmom,dolvol,idiovol,indmom,...,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2,bm,bm_ia,"lead(RET,1)"
DATE,,,,,,,,,,,,,,,,,,,,,
2010-01-29,86098.0,45513.7500,0.6699,60685.0,0.5189,0.2692,-1.8095,9.3795,0.1002,0.6411,...,6.4700e-07,0.0789,0.0309,0.8049,0.5787,1.0600e-07,73.0,1.0539,0.1835,-0.0696
2010-01-29,90605.0,289238.0061,16.8400,15980.0,1.0500,1.1025,-0.0257,11.2450,0.0797,0.0841,...,3.8500e-08,0.0899,0.0261,1.1221,2.4186,4.1400e-08,42.0,0.7493,-0.3943,-0.0075
2010-01-29,63706.0,283082.7930,30.5700,9191.0,0.9752,0.9509,-0.6351,11.2540,0.0463,0.3997,...,2.7900e-08,0.0109,0.0119,0.5471,0.7024,8.2600e-08,67.0,0.0003,-1.5171,0.0000
2010-01-29,38746.0,82145.1989,13.7000,5996.0,1.3773,1.8970,0.8894,9.4867,0.0783,0.6636,...,4.6000e-07,0.0415,0.0157,1.0795,0.7888,9.3700e-08,65.0,0.5191,-1.5351,0.0142
2010-01-29,89898.0,83790.6580,1.4300,63428.0,0.8979,0.8063,0.3844,10.9682,0.1213,0.1358,...,7.7000e-08,0.0966,0.0300,0.5177,2.0774,2.5100e-08,44.0,0.1074,-1.1079,-0.0436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31,20123.0,294300.0126,10.3600,30000.0,1.0233,1.0505,-0.0134,13.1705,0.0431,-0.0195,...,8.3500e-09,0.0143,0.0077,1.6162,2.2928,1.5500e-07,55.0,0.5873,-0.1299,0.1974
2020-12-31,79150.0,537326.9892,45.2600,14215.0,1.1219,1.2586,0.5472,13.4081,0.0673,0.3322,...,8.2500e-09,0.0920,0.0407,0.4779,2.8344,1.6400e-08,28.0,0.5873,-0.1299,-0.0541
2020-12-31,17444.0,30115.7999,13.2800,2145.0,1.8580,3.4522,0.5637,8.8356,0.0818,0.3176,...,1.1500e-05,0.0668,0.0318,1.5229,0.8889,9.9200e-08,36.0,0.5873,-0.1299,0.1030


In [ ]:
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
      df.drop(['lead(RET,1)'], axis=1),
      df.loc[:,['lead(RET,1)']],
      test_size=0.3, shuffle=False)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((54484, 99), (54484, 1), (23351, 99), (23351, 1))

In [ ]:
model_lin = LinearRegression()

In [ ]:
%time model_lin.fit(X_train, y_train)

CPU times: user 758 ms, sys: 51.4 ms, total: 810 ms
Wall time: 462 ms


LinearRegression()

In [ ]:
### COMPUTE STOCHASTIC DROPOUT ###
%%time
#mse
msescores = []
for i in tqdm.tqdm(range(0,50)):
    msescores.append(mean_squared_error(y_test, model_lin.predict(X_test).ravel()))

#mae
maescores = []
for i in tqdm.tqdm(range(0,50)):
    maescores.append(mean_absolute_error(y_test, model_lin.predict(X_test).ravel()))

100%|██████████| 50/50 [00:00<00:00, 103.14it/s]

CPU times: user 1.16 s, sys: 667 ms, total: 1.82 s
Wall time: 1 s


In [ ]:
print(np.mean(msescores), np.std(msescores),np.mean(maescores), np.std(maescores))

0.033583998177776214 6.938893903907228e-18 0.10290062634000736 1.3877787807814457e-17
